In [ ]:
import time
import qsharp
from qsharp_widgets import Circuit, Histogram

import qiskit_aer
import qiskit_aer.noise

from qsharp.noisy_simulator.qir_runner import Simulator, to_results, QirOps
from qsharp.noisy_simulator.noise_model import create_default_noise_model, amplitude_damping_kraus

qsharp.init(target_profile=qsharp.TargetProfile.Base)

In [ ]:
%%qsharp

operation MyH(q: Qubit) : Unit {
  Rz(3.141592653589793 / 2.0, q);
  SX(q);
  Rz(3.141592653589793 / 2.0, q);
}

operation MyCX(control: Qubit, target: Qubit) : Unit {
  MyH(target);
  CZ(control, target);
  MyH(target);
}

operation Main() : Result[] {
  use q = Qubit[2];

  MyH(q[0]);
  MyCX(q[0], q[1]);

  return MResetEachZ(q);
}

In [ ]:
Circuit(qsharp.circuit(qsharp.code.Main))

In [ ]:
qsharp.run(qsharp.code.Main, shots=10)

In [ ]:
qir = qsharp.compile(qsharp.code.Main)

noise = create_default_noise_model()
sim = Simulator(noise_model=noise)
results = sim.run(qir, shots=10)
print("\n".join(results))

In [ ]:
noise.add_kraus_operator("amp_damp_015", amplitude_damping_kraus(0.15))
noise.update_gate_noise("sx", "amp_damp_015")
noise.update_gate_noise("rz", "amp_damp_015")

results = sim.run(qir, shots=10)
print("\n".join(results))

In [ ]:
qsharp.init(target_profile=qsharp.TargetProfile.Base)

In [ ]:
%%qsharp

/// # Sample
/// Simulation of a simple Ising model evolution
/// on a 2D grid with second-order Trotterization.
///
/// # Description
/// This sample demonstrates simulation of an Ising model Hamiltonian
/// on N1xN2 2D grid using a second-order Trotter-Suzuki approximation.
/// This sample can be easily simulated classically with 3x3 grid and
/// about 1000 shots. This sample is suitable for Base Profile.
/// For the purpose of simplicity this sample intentionally doesn't
/// post-process results or perform eigenvalue estimation.
operation Main() : Result[] {
    // Dimensions of a 2D grid is N1 x N2
    let N1 : Int = 3;
    let N2 : Int = 4;

    // Total evolution time
    let evolutionTime : Double = 4.0;
    // Number of steps
    let numberOfSteps : Int = 5;

    // Coefficient for 2-qubit interactions between neighboring qubits
    let J : Double = 1.0;
    // Coefficient for external field interaction for individual qubits
    let g : Double = 1.4;

    // Also try simulating with different strength of external field:
    // let g = 0.2;
    // let g = 1.0;
    // let g = 1.4;
    // let g = 2.0;

    // Model evolution
    IsingModel2DEvolution(N1, N2, J, g, evolutionTime, numberOfSteps)
}

/// # Summary
/// Simulate simple Ising model evolution
///
/// # Description
/// Simulates state |𝜓⟩ evolution to find |𝜓(t)⟩=U(t)|𝜓(0)⟩.
/// |𝜓(0)⟩ is taken to be |0...0⟩.
/// U(t)=e⁻ⁱᴴᵗ, where H is an Ising model Hamiltonian H = -J·Σ'ᵢⱼZᵢZⱼ + g·ΣᵢXᵢ
/// Here Σ' is taken over all pairs of neighboring qubits <i,j>.
/// Simulation is done by performing K steps assuming U(t)≈(U(t/K))ᴷ.
operation IsingModel2DEvolution(
    N1 : Int,
    N2 : Int,
    J : Double,
    g : Double,
    evolutionTime : Double,
    numberOfSteps : Int
) : Result[] {

    // Allocate qubit grid and structure it as a 2D array.
    use qubits = Qubit[N1 * N2];
    let qubitsAs2D = Std.Arrays.Chunks(N2, qubits);

    // Compute the time step
    let dt : Double = evolutionTime / Std.Convert.IntAsDouble(numberOfSteps);

    let theta_x = - g * dt;
    let theta_zz = J * dt;

    // Perform K steps
    for i in 1..numberOfSteps {

        // Single-qubit interaction with external field. Half-step.
        for q in qubits {
            Rx(theta_x, q);
        }

        // All Rzz gates applied in the following two loops commute so they can be
        // applied in any order. To reduce the depth of the algorithm, Rzz gates
        // between horizontal "even" pairs of qubits are applied first - pairs
        // that start at even indices. Then Rzz gates between "odd" pairs are
        // applied. That way all Rzz between horizontal "even" pairs can potentially
        // be done in parallel. Same is true about horizontal "odd"  pairs,
        // vertical "even" pairs and vertical "odd" pairs.

        // Horizontal two-qubit interactions.
        for row in 0..N1-1 {
            // Horizontal interactions between "even" pairs
            for col in 0..2..N2-2 {
                Rzz(2.0 * theta_zz, qubitsAs2D[row][col], qubitsAs2D[row][col + 1]);
            }

            // Horizontal interactions between "odd" pairs
            for col in 1..2..N2-2 {
                Rzz(2.0 * theta_zz, qubitsAs2D[row][col], qubitsAs2D[row][col + 1]);
            }
        }

        // Vertical two-qubit interactions
        for col in 0..N2-1 {

            // Vertical interactions between "even" pairs
            for row in 0..2..N1-2 {
                Rzz(2.0 * theta_zz, qubitsAs2D[row][col], qubitsAs2D[row + 1][col]);
            }

            // Vertical interactions between "odd" pairs
            for row in 1..2..N1-2 {
                Rzz(2.0 * theta_zz, qubitsAs2D[row][col], qubitsAs2D[row + 1][col]);
            }

        }

        // Single-qubit interaction with external field. Half-step.
        for q in qubits {
            Rx(theta_x, q);
        }

    }

    MResetEachZ(qubits)
}


In [ ]:
circ_data = qsharp.circuit(qsharp.code.Main)
Circuit(circ_data)

In [ ]:
start = time.time()
results = qsharp.run(qsharp.code.Main, shots=10)
end = time.time()
print(f"Execution time: {end - start:.2f} seconds")

Histogram(results)

In [ ]:
ising = qsharp.compile(qsharp.code.Main)
prog = QirOps(ising)
prog.transpose()
display(Circuit(prog))

In [ ]:
# Default noise model predefines common gates but without any noise
noise = create_default_noise_model()

# Add amplitude damping noise to the noise model
noise.add_kraus_operator("amp_damp_015", amplitude_damping_kraus(0.01))
noise.update_gate_noise("sx", "amp_damp_015")
noise.update_gate_noise("rz", "amp_damp_015")

# Let's assume moving stuff into the zone for CZ gates is lossy too
noise.update_gate_loss("cz", 0.005)

# The noisy simulator runs on QIR, so compile to QIR first.
sim = Simulator(noise_model=noise)

start = time.time()
results = to_results(sim.run(ising, shots=10))
end = time.time()
print(f"Execution time: {end - start:.2f} seconds")

Histogram(results)

In [ ]:
circ = prog.to_qiskit_circuit()

error = qiskit_aer.noise.amplitude_damping_error(0.15)
noise = qiskit_aer.noise.NoiseModel()
noise.add_all_qubit_quantum_error(error, ['sx', 'rz'])

simulator = qiskit_aer.AerSimulator(method='statevector', noise_model=noise)
start = time.time()
result = simulator.run(circ, shots=10, memory=True).result()
memory = result.get_memory(circ)
end = time.time()
print(f"Execution time: {end - start:.2f} seconds")

shots = to_results(memory)
Histogram(shots)